In [ ]:
import os
import shutil
import cv2
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D, GlobalAveragePooling2D, Concatenate, GlobalMaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50

from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2

from keras.metrics import Recall,Precision
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# from keras import models, layers
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix

from IPython.core.interactiveshell import InteractiveShell

In [ ]:
import os
from pathlib import Path
import pandas as pd


def load_dataset_with_parent_labels(root_path):
    T_df = []
    N_df = []
    M_df = []
    
    root_path = Path(root_path)
    # data = []
    
    # Iterate through each parent folder (label)
    for parent_dir in root_path.iterdir():
        if parent_dir.is_dir():  # Ensure it's a directory
            # label = label_dir.name
            if parent_dir.name == "M":
                for subcat in (parent_dir/"Fused_PT_CT").iterdir():
                    label = subcat.name.split("-")[-1] # getting the sub class name
                    for img_path in subcat.glob('*.png'):
                        # Skip ground truth images (ending with 'GT')
                        if not img_path.stem.endswith('GT'):
                            M_df.append({
                                'path': str(img_path),
                                'label': label
                            })
                M_df = pd.DataFrame(M_df)            
            
            if parent_dir.name == 'T':
                    for subcat in parent_dir.iterdir():
                        label = subcat.name.split("-")[-1] # getting the sub class name
                        if label in ["T1a", "T1c", "T4"] or 'Fused' in subcat.name:
                        
                            for img_path in subcat.glob('*.png'):
                                # Skip ground truth images (ending with 'GT')
                                if not img_path.stem.endswith('GT'):
                                    T_df.append({
                                        'path': str(img_path),
                                        'label': label
                                    })
                    T_df = pd.DataFrame(T_df)            
            else:
                
                for subcat in parent_dir.iterdir():
                    if 'Fused' in subcat.name:
                        label = subcat.name.split("-")[-1]
                        for img_path in subcat.glob('*.png'):
                        # Skip ground truth images (ending with 'GT')
                            if not img_path.stem.endswith('GT'):
                                N_df.append({
                                    'path': str(img_path),
                                    'label': label
                                })
                N_df = pd.DataFrame(N_df)
                            
    return T_df, M_df, N_df
    
root_path = "/kaggle/input/augmented-fusion-new/aug images"
T_df, M_df, N_df = load_dataset_with_parent_labels(root_path)
# Verify
print(f"Total images: {len(T_df)}")
print("Label distribution:")
print("\n",T_df['label'].value_counts())

print(f"Total images: {len(M_df)}")
print("Label distribution:")
print("\n",M_df['label'].value_counts())

print(f"Total images: {len(N_df)}")
print("Label distribution:")
print(N_df['label'].value_counts())

# M Stage

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocessing


M_train_df, M_test_df = train_test_split(M_df, test_size=0.2, random_state=2, shuffle=True, stratify=M_df['label'])

# debugging_df, _ = train_test_split(
#     train_df,
#     train_size=1,  # 10% of the data
#     stratify=train_df['label'],  # Maintain class distribution
#     random_state=42,
#     shuffle=True
# )
batch_size = 32
train_generator = ImageDataGenerator(
    preprocessing_function=eff_preprocessing,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest', 
    validation_split=0.2
)
test_generator = ImageDataGenerator(preprocessing_function=eff_preprocessing)

M_train_imgs = train_generator.flow_from_dataframe(
    dataframe = M_train_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True,
    subset = "training"
)

M_val_imgs = train_generator.flow_from_dataframe(
    dataframe = M_train_df,
    x_col = "path",
    y_col = "label",
    target_size =(224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False,
    subset = "validation"
)

M_test_imgs = test_generator.flow_from_dataframe(
    dataframe = M_test_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False
)  # No augmentation for validation

# T Stage

In [ ]:
from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocessing


T_train_df, T_test_df = train_test_split(T_df, test_size=0.2, random_state=2, shuffle=True, stratify=T_df['label'])

batch_size = 32
train_generator = ImageDataGenerator(
    preprocessing_function=eff_preprocessing,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest', 
    validation_split=0.2
)
test_generator = ImageDataGenerator(preprocessing_function=eff_preprocessing)

T_train_imgs = train_generator.flow_from_dataframe(
    dataframe = T_train_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True,
    subset = "training"
)

T_val_imgs = train_generator.flow_from_dataframe(
    dataframe = T_train_df,
    x_col = "path",
    y_col = "label",
    target_size =(224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False,
    subset = "validation"
)

T_test_imgs = test_generator.flow_from_dataframe(
    dataframe = T_test_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False
)  # No augmentation for validation

# N Stage

In [ ]:
N_train_df, N_test_df = train_test_split(N_df, test_size=0.2, random_state=2, shuffle=True, stratify=N_df['label'])

batch_size = 32
train_generator = ImageDataGenerator(
    preprocessing_function=eff_preprocessing,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest', 
    validation_split=0.2
)
test_generator = ImageDataGenerator(preprocessing_function=eff_preprocessing)

N_train_imgs = train_generator.flow_from_dataframe(
    dataframe = N_train_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = True,
    subset = "training"
)

N_val_imgs = train_generator.flow_from_dataframe(
    dataframe = N_train_df,
    x_col = "path",
    y_col = "label",
    target_size =(224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False,
    subset = "validation"
)

N_test_imgs = test_generator.flow_from_dataframe(
    dataframe = N_test_df,
    x_col = "path",
    y_col = "label",
    target_size = (224,224),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = batch_size,
    shuffle = False
)  # No augmentation for validation

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import mixed_precision
from tensorflow.keras.applications import EfficientNetB0, DenseNet121, DenseNet201, ResNet50
from tensorflow.keras.models import Model


def build_model(class_count, with_gradcam=False):
    base_model = EfficientNetB0(
        input_shape=(224, 224, 3),
        include_top=False,
        weights="imagenet",
    )
    base_model.trainable = False
    for layer in base_model.layers[-30:]: #30   reduce the unfreezed layers -->  less complex model
        layer.trainable = True
    
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(256)(x)
    x = BatchNormalization()(x)  
    x = Activation('relu')(x)
    
    x = Dense(256)(x)
    x = BatchNormalization()(x)   
    x = Activation('relu')(x)   

    x = Dense(128)(x)
    x = Activation('relu')(x)
    
    x = Dense(64)(x)
    x = Activation('relu')(x)
    
    classification_output = Dense(class_count, activation='softmax', name='cls_output')(x)

    if with_gradcam:
        # Output both predictions and last conv layer features
        model = Model(
            inputs=base_model.input,
            outputs={'cls_output':classification_output, 
                     'vis_layer':base_model.get_layer('top_conv').output}
        )
    else:
        # Standard classification model
        model = Model(inputs=base_model.input,outputs=classification_output)
    
    return model

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

def evaluate(model, test_imgs, history, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    true_labels = test_imgs.classes
    pred_probs = model.predict(test_imgs)['cls_output']
    pred_labels = np.argmax(pred_probs, axis=1)
    class_labels = list(test_imgs.class_indices.keys())

    report = classification_report(true_labels, pred_labels, target_names=class_labels)
    print(report)
    with open(os.path.join(output_dir, "classification_report.txt"), "w") as f:
        f.write(report)

    plt.figure(figsize=(12, 5))
    
    # Accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['cls_output_accuracy'], label='Train Accuracy', marker='o')
    plt.plot(history.history['val_cls_output_accuracy'], label='Validation Accuracy', marker='x')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, "acc.png"))
    plt.show()
    plt.close()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss', marker='o')
    plt.plot(history.history['val_loss'], label='Validation Loss', marker='x')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "loss.png"))
    plt.show()
    plt.close()

    # Confusion matrix
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels, cmap='Blues')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.title('Confusion Matrix')

    cm_plot_path = os.path.join(output_dir, "confusion_matrix.png")
    plt.savefig(cm_plot_path)
    plt.close()

    print(f"\nSaved classification report, plots to '{output_dir}'")


In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, output_index=0):
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(last_conv_layer_name).output, model.output[output_index] if isinstance(model.output, (list, tuple)) else model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions['cls_output'])
        class_output = predictions['cls_output']

    grads = tape.gradient(class_output, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap + tf.keras.backend.epsilon())

    return heatmap.numpy()


In [ ]:
import cv2
import matplotlib.pyplot as plt
def display_gradcam_on_image(img_array, original_img, heatmap, true_label, predicted_label, alpha=0.4):

    heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    superimposed_img = heatmap_color * alpha + original_img
    superimposed_img = np.uint8(superimposed_img)

    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"{true_label} Class")
    plt.show()

# Pre-training the Model on M Stage

In [ ]:
# train_imgs, val_imgs = load_and_preprocess_data()
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.optimizers.schedules import ExponentialDecay

training_samples_num = len(M_train_df)*0.8

initial_learning_rate = 5e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=(training_samples_num//batch_size)*2,
    decay_rate=0.4, # reduce by 20%
    staircase=True)

M_model = build_model(4, with_gradcam=True)

M_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss={'cls_output':'categorical_crossentropy'},
    metrics=['accuracy']
)

class LrValLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, 'numpy'):
            lr_value = lr.numpy()
        else:
            lr_value = tf.keras.backend.get_value(lr)
        
        val_loss = logs.get('val_loss')
        print(f"            learning rate ={lr_value:.6f}")

M_history = M_model.fit(
    M_train_imgs,
    validation_data=M_val_imgs,
    epochs=15,
    callbacks=[
        LrValLogger()
    ]
)

In [ ]:
os.makedirs('output', exist_ok=True)
M_model.save('output/M_model.h5')

In [ ]:
evaluate(M_model, M_test_imgs, M_history, output_dir="evaluation_results_M")

In [ ]:
class_indices = M_train_imgs.class_indices
classes = list(class_indices.keys())

X_batch, y_batch = next(M_train_imgs)
vis_classes = []

for i in range(min(20, len(X_batch))):
    img_array = np.expand_dims(X_batch[i], axis=0)
    true_idx = np.argmax(y_batch[i])
    true_label = classes[true_idx]

    if true_label in vis_classes:
        continue
        
    vis_classes.append(true_label)
    
    # Predict using the model
    preds = M_model.predict(img_array)['cls_output']
    
    # If model has multiple outputs, select correct head
    if isinstance(preds, (list, tuple)):
        preds = preds[0]  # or whatever head you care about

    pred_idx = np.argmax(preds[0])
    predicted_label = classes[pred_idx]

    if true_idx != pred_idx:
        print(f"Skipped (True: {true_label}, Pred: {predicted_label})")
        continue

    # Grad-CAM
    heatmap = make_gradcam_heatmap(
        img_array, M_model, last_conv_layer_name="top_conv", output_index=0
    )

    # Visualize
    original_img = X_batch[i]
    original_img_disp = np.uint8((original_img - original_img.min()) / (original_img.max() - original_img.min()) * 255)

    display_gradcam_on_image(img_array, original_img_disp, heatmap, true_label, predicted_label)


# Pre-training the model on N Stage

In [ ]:
training_samples_num = len(N_train_df)*0.8

initial_learning_rate = 5e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=(training_samples_num//batch_size)*2,
    decay_rate=0.4,
    staircase=True)

N_model = build_model(3, with_gradcam=True)

N_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss={'cls_output':'categorical_crossentropy'},
    metrics=['accuracy']
)

class LrValLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, 'numpy'):
            lr_value = lr.numpy()
        else:
            lr_value = tf.keras.backend.get_value(lr)
        
        val_loss = logs.get('val_loss')
        print(f"            learning rate ={lr_value:.6f}")

N_history = N_model.fit(
    N_train_imgs,
    validation_data=N_val_imgs,
    epochs=15,
    callbacks=[
        LrValLogger()
    ]
)

In [ ]:
N_model.save('output/N_model.h5')

In [ ]:
evaluate(M_model, N_test_imgs, N_history, output_dir="evaluation_results_N")

In [ ]:
class_indices = N_train_imgs.class_indices
classes = list(class_indices.keys())

X_batch, y_batch = next(N_train_imgs)
vis_classes = []

for i in range(min(20, len(X_batch))):
    img_array = np.expand_dims(X_batch[i], axis=0)
    true_idx = np.argmax(y_batch[i])
    true_label = classes[true_idx]

    if true_label in vis_classes:
        continue
        
    vis_classes.append(true_label)
    
    # Predict using the model
    preds = N_model.predict(img_array)['cls_output']
    
    # If model has multiple outputs, select correct head
    if isinstance(preds, (list, tuple)):
        preds = preds[0]  # or whatever head you care about

    pred_idx = np.argmax(preds[0])
    predicted_label = classes[pred_idx]

    if true_idx != pred_idx:
        print(f"Skipped (True: {true_label}, Pred: {predicted_label})")
        continue

    # Grad-CAM
    heatmap = make_gradcam_heatmap(
        img_array, N_model, last_conv_layer_name="top_conv", output_index=0
    )

    # Visualize
    original_img = X_batch[i]
    original_img_disp = np.uint8((original_img - original_img.min()) / (original_img.max() - original_img.min()) * 255)

    display_gradcam_on_image(img_array, original_img_disp, heatmap, true_label, predicted_label)


# Pre-training the model on T stage

In [ ]:
training_samples_num = len(Ttrain_df)*0.8

initial_learning_rate = 5e-3
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=(training_samples_num//batch_size)*2,
    decay_rate=0.4,
    staircase=True)

T_model = build_model(3, with_gradcam=True)

T_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss={'cls_output':'categorical_crossentropy'},
    metrics=['accuracy']
)

class LrValLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        lr = self.model.optimizer.learning_rate
        if hasattr(lr, 'numpy'):
            lr_value = lr.numpy()
        else:
            lr_value = tf.keras.backend.get_value(lr)
        
        val_loss = logs.get('val_loss')
        print(f"            learning rate ={lr_value:.6f}")

T_history = T_model.fit(
    T_train_imgs,
    validation_data=T_val_imgs,
    epochs=15,
    callbacks=[
        LrValLogger()
    ]
)

In [ ]:
T_model.save('output/N_model.h5')

In [ ]:
evaluate(T_model, T_test_imgs, T_history, output_dir="evaluation_results_T")

In [ ]:
class_indices = T_train_imgs.class_indices
classes = list(class_indices.keys())

X_batch, y_batch = next(T_train_imgs)
vis_classes = []

for i in range(min(20, len(X_batch))):
    img_array = np.expand_dims(X_batch[i], axis=0)
    true_idx = np.argmax(y_batch[i])
    true_label = classes[true_idx]

    if true_label in vis_classes:
        continue
        
    vis_classes.append(true_label)
    
    # Predict using the model
    preds = T_model.predict(img_array)['cls_output']
    
    # If model has multiple outputs, select correct head
    if isinstance(preds, (list, tuple)):
        preds = preds[0]  # or whatever head you care about

    pred_idx = np.argmax(preds[0])
    predicted_label = classes[pred_idx]

    if true_idx != pred_idx:
        print(f"Skipped (True: {true_label}, Pred: {predicted_label})")
        continue

    # Grad-CAM
    heatmap = make_gradcam_heatmap(
        img_array, T_model, last_conv_layer_name="top_conv", output_index=0
    )

    # Visualize
    original_img = X_batch[i]
    original_img_disp = np.uint8((original_img - original_img.min()) / (original_img.max() - original_img.min()) * 255)

    display_gradcam_on_image(img_array, original_img_disp, heatmap, true_label, predicted_label)
